In [24]:
import ollama
import json
import numpy as np
from sentence_transformers import SentenceTransformer

In [21]:
?ollama.embed

Signature:
ollama.embed(
    model: str = '',
    input: Union[str, Sequence[str]] = '',
    truncate: Optional[bool] = None,
    options: Union[Mapping[str, Any], ollama._types.Options, NoneType] = None,
    keep_alive: Union[float, str, NoneType] = None,
) -> ollama._types.EmbedResponse
Docstring: <no docstring>
File:      c:\users\ariji\onedrive\documents\code\2025\.venv\lib\site-packages\ollama\_client.py
Type:      method

In [35]:
embeddings = ollama.embed(
        model='mxbai-embed-large:latest',
        input=['Llamas are members of the camelid family.','Goku is a super saiyan']
    )
# print as a pretty json
np_emb  = np.array(embeddings['embeddings'])


In [36]:
#print(json.dumps(embeddings['embeddings'], indent=4))

print(np_emb.shape)

(2, 1024)


In [37]:
#?mxb_encoder.encode

In [34]:
mxb_encoder = SentenceTransformer(r'../models/mixedbread-ai/mxbai-embed-large-v1', truncate_dim=512)
mxb_emb = mxb_encoder.encode(['Llamas are members of the camelid family.','Goku is a super saiyan'], convert_to_numpy=True)
print((mxb_emb).shape)


(2, 512)


Storing embeddings in vector database

In [38]:
import chromadb

In [39]:
documents = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
]

In [44]:
client = chromadb.Client()

In [45]:
?client.get_or_create_collection

Signature:
client.get_or_create_collection(
    name: str,
    configuration: Optional[chromadb.api.configuration.CollectionConfigurationInterface] = None,
    metadata: Optional[Dict[str, Any]] = None,
    embedding_function: Optional[chromadb.api.types.EmbeddingFunction[Union[List[str], List[numpy.ndarray[Any, numpy.dtype[Union[numpy.uint64, numpy.int64, numpy.float64]]]]]]] = <chromadb.utils.embedding_functions.onnx_mini_lm_l6_v2.ONNXMiniLM_L6_V2 object at 0x00000232442D55E0>,
    data_loader: Optional[chromadb.api.types.DataLoader[List[Optional[numpy.ndarray[Any, numpy.dtype[Union[numpy.uint64, numpy.int64, numpy.float64]]]]]]] = None,
) -> chromadb.api.models.Collection.Collection
Docstring:
Get or create a collection with the given name and metadata.
Args:
    name: The name of the collection to get or create
    metadata: Optional metadata to associate with the collection. If
    the collection already exists, the metadata provided is ignored.
    If the collection does not exis

In [61]:
client.list_collections()

['llama_index2', 'llama_index']

In [46]:
#client = chromadb.Client()
collection = client.get_or_create_collection(name='llama_index')


In [50]:
for index, document in enumerate(documents):
    response = ollama.embed(model="mxbai-embed-large", input=document)
    embeddings = response['embeddings']
    collection.add(
        # ids=str(index),
        ids=[str(index)],
        embeddings=embeddings,
        documents=[document]
    )


In [72]:
documents_2 = [
  "Llamas are members of the camelid family meaning they're pretty closely related to vicuñas, alpacas and camels",
  "Llamas were first domesticated and used as pack animals 4,000 to 5,000 years ago in the Peruvian highlands",
  "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
  "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
  "Llamas are vegetarians and have very efficient digestive systems",
  "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
  "Goku is a super saiyan",
  "Goku has a pet llama",
  "Goku's pet llama is Vicuna",
  "Vegeta is the prince of Saiyans",
  "The word Vegeta is a pun on vegetables",
  "Brinjal is my favorite vegetable",
  "Garima loves alpacas"

]
collection_2 = client.get_or_create_collection(name='llama_index2')
for index, document in enumerate(documents_2):
    response = ollama.embed(model="mxbai-embed-large", input=document)
    embeddings = response['embeddings']
    collection.add(
        ids=str(index),
        embeddings=embeddings,
        documents=[document]
    )


Add of existing embedding ID: 0
Insert of existing embedding ID: 0
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 3
Add of existing embedding ID: 4
Insert of existing embedding ID: 4
Add of existing embedding ID: 5
Insert of existing embedding ID: 5
Add of existing embedding ID: 6
Insert of existing embedding ID: 6
Add of existing embedding ID: 7
Insert of existing embedding ID: 7
Add of existing embedding ID: 8
Insert of existing embedding ID: 8


In [81]:
# an example input
input = "What animals are llamas related to?"
input2 = "What do you know about Vegeta?"
# generate an embedding for the input and retrieve the most relevant doc
response = ollama.embed(
  model="mxbai-embed-large",
  input=input2
)

In [82]:
results = collection.query(
  query_embeddings=response["embeddings"],
  n_results=10
)


In [83]:
print(json.dumps(results, indent=4))

{
    "ids": [
        [
            "9",
            "10",
            "6",
            "7",
            "8",
            "2",
            "4",
            "5",
            "3",
            "11"
        ]
    ],
    "embeddings": null,
    "documents": [
        [
            "Vegeta is the prince of Saiyans",
            "The word Vegeta is a pun on vegetables",
            "Goku is a super saiyan",
            "Goku has a pet llama",
            "Goku's pet llama is Vicuna",
            "Llamas can grow as much as 6 feet tall though the average llama between 5 feet 6 inches and 5 feet 9 inches tall",
            "Llamas are vegetarians and have very efficient digestive systems",
            "Llamas live to be about 20 years old, though some only live for 15 years and others live to be 30 years old",
            "Llamas weigh between 280 and 450 pounds and can carry 25 to 30 percent of their body weight",
            "Brinjal is my favorite vegetable"
        ]
    ],
    "uris": nul